In [4]:
# Autoreload possibly interferes with IntelliJ debugging
%reload_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: logging.info(msg)


# Full pipeline (multiple files)

In [5]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")

# Post-human processing
This requires user interaction first to provide sleep times etc.

In [2]:
from sleep_events import load_days_data
from sleep_events import pimp_my_days_data

days_data = load_days_data()
days_data = pimp_my_days_data(days_data)

In [30]:
import run_feature_pipeline
from run_post_human_pipeline import cached_post_human_pipeline

#dir_name = "2024-11-01-21-32-22"
dir_name = "2024-11-07-21-26-18"
input_file = f"C:\\dev\\play\\brainwave-data\\{dir_name}\\raw.fif"
yasa_df = run_feature_pipeline.cached_pipeline(log, input_file, stats_df)
post_human_df = cached_post_human_pipeline(log, dir_name, input_file, stats_df, days_data, yasa_df)
post_human_df

C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:77: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-09 20:17:27.992000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:79: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']
100%|██████████| 1245/1245 [00:08<00:00, 149.63it/s]


array([[0.96418475, 0.03581525],
       [0.98562164, 0.01437836],
       [0.98999225, 0.01000775],
       ...,
       [0.59728338, 0.40271662],
       [0.56832644, 0.43167356],
       [0.6313277 , 0.3686723 ]])

# Replicate the pipeline

In [57]:
from models.eeg_states.eeg_states import load_and_prepare_eeg_state_events

eeg_state_events = load_and_prepare_eeg_state_events()


C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-09 20:17:27.992000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  return events
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:83: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  


In [58]:
eeg_state_events

,timestamp,event,since,duration,TimestampUK,SinceUK,dayAndNightOf
0,2024-10-09 20:22:27.992000+00:00,tired_long,2024-10-09 20:17:27.992000+00:00,0 days 00:05:00,2024-10-09 21:22:27.992000+01:00,2024-10-09 21:17:27.992000+01:00,2024-10-09
1,2024-10-09 20:28:57.046000+00:00,tired,NaT,NaT,2024-10-09 21:28:57.046000+01:00,NaT,2024-10-09
2,2024-10-09 20:32:36.824000+00:00,tired,NaT,NaT,2024-10-09 21:32:36.824000+01:00,NaT,2024-10-09
3,2024-10-09 20:32:52.775000+00:00,tired_long,2024-10-09 20:32:36.824000+00:00,0 days 00:00:15.951000,2024-10-09 21:32:52.775000+01:00,2024-10-09 21:32:36.824000+01:00,2024-10-09
4,2024-10-09 20:33:13.921000+00:00,tired,NaT,NaT,2024-10-09 21:33:13.921000+01:00,NaT,2024-10-09
...,...,...,...,...,...,...,...
232,2024-11-09 21:32:48.749000+00:00,wired,NaT,NaT,2024-11-09 21:32:48.749000+00:00,NaT,2024-11-09
233,2024-11-09 21:33:36.388000+00:00,wired_long,2024-11-09 21:32:48.749000+00:00,0 days 00:00:47.639000,2024-11-09 21:33:36.388000+00:00,2024-11-09 21:32:48.749000+00:00,2024-11-09
234,2024-11-09 21:35:54.060000+00:00,wired_long,2024-11-09 21:33:36.388000+00:00,0 days 00:02:17.672000,2024-11-09 21:35:54.060000+00:00,2024-11-09 21:33:36.388000+00:00,2024-11-09
235,2024-11-09 21:37:25.486000+00:00,wired_long,2024-11-09 21:35:54.060000+00:00,0 days 00:01:31.426000,2024-11-09 21:37:25.486000+00:00,2024-11-09 21:35:54.060000+00:00,2024-11-09


In [61]:
from catboost import CatBoostClassifier
from models.eeg_states.eeg_states_model import predict_only_model_pipeline
from models.eeg_states.eeg_states import load_and_prepare_eeg_state_events, add_event_type
from run_post_human_pipeline import add_periods
from sleep_events import convert_timestamps_to_uk

convert_timestamps_to_uk(yasa_df, 'Timestamp', 'TimestampUK')
yasa_df = add_periods(log, dir_name, input_file, stats_df, days_data, yasa_df)
add_event_type(yasa_df, eeg_state_events)

100%|██████████| 1245/1245 [00:00<00:00, 1498.73it/s]


In [50]:
yasa_df['dayAndNightOf'].info()

<class 'pandas.core.series.Series'>
Index: 1245 entries, 0 to 1244
Series name: dayAndNightOf
Non-Null Count  Dtype 
--------------  ----- 
1245 non-null   object
dtypes: object(1)
memory usage: 51.7+ KB


In [32]:
before_ready_to_sleep = yasa_df[yasa_df['60MinsBeforeReadyToSleep'] == 1]
before_ready_to_sleep.shape

(120, 463)

In [33]:
before_ready_to_sleep

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,minsSinceAsleep,15MinsBeforeReadyToSleep,30MinsBeforeReadyToSleep,60MinsBeforeReadyToSleep,DuringReadyToSleep,minsSinceReadyToSleep,minsSinceGotIntoBed,minsUntilWake,epoch_type,matched_event
epoch,,,,,,,,,,,,,,,,,,,,,
28,W,0.997181,28,2024-11-07 21:40:18.485414982+00:00,Fpz-M1,W,0.997181,0,7.401267e-10,0.161176,...,-122.19191,0,0,1,0,-59.69191,12.30809,583.49191,None,None
29,W,0.997597,29,2024-11-07 21:40:48.485414982+00:00,Fpz-M1,W,0.997597,0,2.476341e-09,0.153376,...,-121.69191,0,0,1,0,-59.19191,12.80809,582.99191,None,None
30,W,0.998153,30,2024-11-07 21:41:18.485414982+00:00,Fpz-M1,W,0.998153,0,1.169741e-09,0.144437,...,-121.19191,0,0,1,0,-58.69191,13.30809,582.49191,None,None
31,W,0.998330,31,2024-11-07 21:41:48.485414982+00:00,Fpz-M1,W,0.998330,0,4.150246e-10,0.154671,...,-120.69191,0,0,1,0,-58.19191,13.80809,581.99191,None,None
32,W,0.999058,32,2024-11-07 21:42:18.485414982+00:00,Fpz-M1,W,0.999058,0,5.591255e-10,0.172438,...,-120.19191,0,0,1,0,-57.69191,14.30809,581.49191,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,W,0.994437,143,2024-11-07 22:37:48.485414982+00:00,Fpz-M1,W,0.994437,0,1.083327e-09,1.040992,...,-64.69191,1,1,1,0,-2.19191,69.80809,525.99191,None,None
144,W,0.990811,144,2024-11-07 22:38:18.485414982+00:00,Fpz-M1,W,0.990811,0,6.394515e-10,1.468478,...,-64.19191,1,1,1,0,-1.69191,70.30809,525.49191,None,None
145,W,0.982737,145,2024-11-07 22:38:48.485414982+00:00,Fpz-M1,W,0.982737,0,4.419872e-10,1.895721,...,-63.69191,1,1,1,0,-1.19191,70.80809,524.99191,None,None


In [40]:
models_and_data = [predict_only_model_pipeline('main', before_ready_to_sleep)]
model = CatBoostClassifier()
model.load_model("models/eeg_states/main_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
predictions_df['TiredVsWired'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)

In [41]:
predictions_df

,TiredVsWired
epoch,
28,0.315348
29,0.432130
30,0.751347
31,0.414552
32,0.311561
...,...
143,0.030932
144,0.024494
145,0.140188


In [39]:
predictions_df

,0,1,TiredVsWired
epoch,,,
28,0.684652,0.315348,0.315348
29,0.567870,0.432130,0.432130
30,0.248653,0.751347,0.751347
31,0.585448,0.414552,0.414552
32,0.688439,0.311561,0.311561
...,...,...,...
143,0.969068,0.030932,0.030932
144,0.975506,0.024494,0.024494
145,0.859812,0.140188,0.140188


In [42]:
yasa_df_with_predictions = pd.concat([yasa_df, predictions_df], axis=1)

In [43]:
yasa_df_with_predictions

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,15MinsBeforeReadyToSleep,30MinsBeforeReadyToSleep,60MinsBeforeReadyToSleep,DuringReadyToSleep,minsSinceReadyToSleep,minsSinceGotIntoBed,minsUntilWake,epoch_type,matched_event,TiredVsWired
epoch,,,,,,,,,,,,,,,,,,,,,
0,W,0.999395,0,2024-11-07 21:26:18.485414982+00:00,Fpz-M1,W,0.999395,0,4.993414e-09,3.034542,...,0,0,0,0,-73.69191,-1.69191,597.49191,wired,210,NaN
1,W,0.999047,1,2024-11-07 21:26:48.485414982+00:00,Fpz-M1,W,0.999047,0,4.687774e-08,2.991824,...,0,0,0,0,-73.19191,-1.19191,596.99191,None,None,NaN
2,W,0.999430,2,2024-11-07 21:27:18.485414982+00:00,Fpz-M1,W,0.999430,0,1.161335e-08,2.729585,...,0,0,0,0,-72.69191,-0.69191,596.49191,None,None,NaN
3,W,0.999343,3,2024-11-07 21:27:48.485414982+00:00,Fpz-M1,W,0.999343,0,2.888595e-08,2.509642,...,0,0,0,0,-72.19191,-0.19191,595.99191,None,None,NaN
4,W,0.998353,4,2024-11-07 21:28:18.485414982+00:00,Fpz-M1,W,0.998353,0,2.170471e-08,2.220646,...,0,0,0,0,-71.69191,0.30809,595.49191,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,W,0.995203,1240,2024-11-08 07:46:18.485414982+00:00,Fpz-M1,W,0.995203,0,1.276919e-09,0.168469,...,0,0,0,0,546.30809,618.30809,-22.50809,None,None,NaN
1241,W,0.997428,1241,2024-11-08 07:46:48.485414982+00:00,Fpz-M1,W,0.997428,0,7.675433e-10,0.161403,...,0,0,0,0,546.80809,618.80809,-23.00809,None,None,NaN
1242,W,0.996014,1242,2024-11-08 07:47:18.485414982+00:00,Fpz-M1,W,0.996014,0,9.583334e-10,0.156325,...,0,0,0,0,547.30809,619.30809,-23.50809,None,None,NaN


In [45]:
yasa_df_with_predictions["TiredVsWired"].mean()

0.5650632491976203